<a href="https://colab.research.google.com/github/zxarda01/Tweet-Sentiment-Classification-/blob/main/SVM_Word2Vec_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Data Preparation
!wget https://raw.githubusercontent.com/whyjay17/Twitter-Sentiment-Analysis/master/data/twitter-2016dev-A.txt


--2020-12-29 17:13:40--  https://raw.githubusercontent.com/whyjay17/Twitter-Sentiment-Analysis/master/data/twitter-2016dev-A.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 294054 (287K) [text/plain]
Saving to: ‘twitter-2016dev-A.txt’

twitter-2016dev-A.t 100%[===================>] 287.16K  --.-KB/s    in 0.04s   

2020-12-29 17:13:41 (7.34 MB/s) - ‘twitter-2016dev-A.txt’ saved [294054/294054]



In [2]:
!wget -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"

--2020-12-29 17:13:45--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.228.115
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.228.115|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1647046227 (1.5G) [application/x-gzip]
Saving to: ‘GoogleNews-vectors-negative300.bin.gz’

GoogleNews-vectors- 100%[===================>]   1.53G  16.9MB/s    in 95s     

2020-12-29 17:15:20 (16.6 MB/s) - ‘GoogleNews-vectors-negative300.bin.gz’ saved [1647046227/1647046227]



In [4]:
# Package Installation
!pip install scikit-learn
!pip install nltk
!pip install numpy
!pip gensim

import nltk
nltk.download('stopwords')

ERROR: unknown command "gensim"
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [40]:
# Data Loading
import csv
import string
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
from gensim.models.keyedvectors import KeyedVectors

tweet_data_path = '/content/twitter-2016dev-A.txt'
word2vec_path = '/content/GoogleNews-vectors-negative300.bin.gz'
tweet_tokenizer = TweetTokenizer()

with open(tweet_data_path , encoding='utf-8') as f:
    reader = csv.reader(f, delimiter="\t")
    tweet_data = list(reader)


print('[input]# of records:',len(tweet_data))
print('[input]sample data:',tweet_data[0])

[input]# of records: 1966
[input]sample data: ['638060586258038784', 'neutral', '05 Beat it - Michael Jackson - Thriller (25th Anniversary Edition) [HD] http://t.co/A4K2B86PBv']


In [42]:
# Data Preprocessing
tweet_data_parsed = []
stop = set(stopwords.words('english'))
for info in tweet_data:
    l = " ".join(tweet_tokenizer.tokenize(info[2].lower())).split(" ")
    filtered_sentence = [w for w in l if not w in stop and not w in string.punctuation
                         and ( w[0] != '@' and w[0] != '#' and w[:4] != 'http' )]
    tweet_data_parsed.append(filtered_sentence)

print('[parsed]# of records:',len(tweet_data_parsed))
print('[parsed]sample data:',tweet_data_parsed[0])

# Label Data
tweet_class = np.zeros(len(tweet_data))
for i in range(len(tweet_data)):
    if tweet_data[i][1] == 'negative':
        tweet_class[i] = 0
    elif tweet_data[i][1] == 'neutral':
        tweet_class[i] = 1
    elif tweet_data[i][1] == 'positive':
        tweet_class[i] = 2

print('[class]# of records:',len(tweet_class))
print('[class]sample data:',tweet_class[0])

[parsed]# of records: 1966
[parsed]sample data: ['05', 'beat', 'michael', 'jackson', 'thriller', '25th', 'anniversary', 'edition', 'hd']
[class]# of records: 1966
[class]sample data: 1.0


In [20]:
print('Model Loading Started')
model = KeyedVectors.load_word2vec_format(word2vec_path, binary=True)
print('Model Loading Finished')

started
finished


In [43]:
# Feature(word2vec) 
tweet_word2vec = []
# adds the word2vec average
for tweet in tweet_data_parsed:
    average_vec = np.zeros(300)
    for word in tweet:
        if word in model.wv:
            average_vec += (model.wv[word] / len(tweet))
        else:
            pass
    tweet_word2vec.append(average_vec)

# train test split
ratio = len(tweet_data) * 0.2
tweet_train = tweet_data_parsed[: -int(ratio)]
tweet_test = tweet_data_parsed[-int(ratio):]

x_train = tweet_word2vec[: -int(ratio)]
y_train = tweet_class[: -int(ratio)]

x_test = tweet_word2vec[-int(ratio):]
y_test = tweet_class[-int(ratio):]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  


In [66]:
from sklearn import preprocessing
x_train = preprocessing.scale(x_train)

# Train SVM
from sklearn import svm

print("started") 

import time
start = time.time()
clf = svm.SVC(decision_function_shape='ovo', verbose=True)
clf.fit(x_train, y_train)
#svm.SVC(decision_function_shape='ovo')

print("finished")
result1 = clf.predict(x_test)

started
[LibSVM]finished


In [67]:
# evaluation

fn_positive = 0
tp_positive = 0

for i, j in zip(y_test, result1):
    if i == 2 and i != j:
        fn_positive += 1
    if i == 2 and i == j:
        tp_positive += 1

fn_neutral = 0
tp_neutral = 0

for i, j in zip(y_test, result1):
    if (i == 1 and i != j):
        fn_neutral += 1
    if i == 1 and i == j:
        tp_neutral += 1

fn_negative = 0
tp_negative = 0

for i, j in zip(y_test, result1):
    if (i == 0 and i != j):
        fn_negative += 1
    if i == 0 and i == j:
        tp_negative += 1

recall_pos = tp_positive / (tp_positive + fn_positive) if (tp_positive + fn_positive) != 0 else 0
recall_neg = tp_negative / (tp_negative + fn_negative) if (tp_negative + fn_negative) != 0 else 0
recall_neu = tp_neutral / (tp_neutral + fn_neutral) if (tp_neutral + fn_neutral) != 0 else 0

print('Average Recall : ', (1/3) * (recall_neg + recall_neu + recall_pos))

Average Recall :  0.49250645994832043


In [64]:
# SVM 
from sklearn.svm import LinearSVC

svc_model = LinearSVC(verbose=1)
svc_model.fit(x_train, y_train)
result1 = svc_model.predict(x_test)

print(len(x_train))
print(len(y_train))

[LibLinear]1573
1573


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
